In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression

In [ ]:
testfiles=['''enter test filenames in this order: comments_employee_mapping, comments_likeability, employee_attrition, happiness_level.csv''']

In [ ]:
mapping=pd.read_csv('comments_employee_mapping.csv').dropna()
likes=pd.read_csv('comments_likeability.csv').dropna()
attr=pd.read_csv('employee_attrition.csv').dropna()
hap=pd.read_csv('happiness_level.csv').dropna()

mapping_test=pd.read_csv(testfiles[0]).dropna()
likes_test=pd.read_csv(testfiles[1]).dropna()
attr_test=pd.read_csv(testfiles[2]).dropna()
hap_test=pd.read_csv(testfiles[3]).dropna()

attr['lastParticipationDate']=pd.to_datetime(attr['lastParticipationDate'],infer_datetime_format=True)
attr_test['lastParticipationDate']=pd.to_datetime(attr_test['lastParticipationDate'],infer_datetime_format=True)
attr['ordinal']=[x.toordinal() for x in attr['lastParticipationDate']]
attr_test['ordinal']=[x.toordinal() for x in attr_test['lastParticipationDate']]

for d in [mapping,likes,attr,hap,mapping_test,likes_test,attr_test,hap_test]:
    d['id']=d['employee'].map(str)+d['companyAlias']

In [ ]:
#Train Preproc
id_dict,comm_dict={},{}
for i in range(len(likes)):
    try: id_dict[likes['id'][i]]==0
    except: id_dict[likes['id'][i]]=len(id_dict.keys())
    try: comm_dict[likes['commentId'][i]]==0
    except: comm_dict[likes['commentId'][i]]=len(comm_dict.keys())

m=np.zeros([len(id_dict.keys()),len(comm_dict.keys())])
for i in range(len(likes)):
    m[id_dict[likes['id'][i]],comm_dict[likes['commentId'][i]]]= likes['liked'][i]*1 - likes['disliked'][i]*1

l=[]
exc=0
for i in range(len(attr)):
    try:
        l.append(m[id_dict[attr['id'][i]],:])
    except:
        exc+=1
        l.append(np.zeros(len(comm_dict)))
l=np.vstack(l)




In [ ]:
#Test PreProc
id_dict_test={}
for i in range(len(likes_test)):
    try: id_dict_test[likes_test['id'][i]]==0
    except: id_dict_test[likes_test['id'][i]]=len(id_dict_test)

m_test=np.zeros([len(id_dict_test.keys()),len(comm_dict.keys())])
for i in range(len(likes_test)):
    try:m_test[id_dict_test[likes_test['id'][i]],comm_dict[likes_test['commentId'][i]]]= likes_test['liked'][i]*1 - likes_test['disliked'][i]*1
    except:pass

l_test=[]
for i in range(len(attr_test)):
    try:l_test.append(m[id_dict_test[attr_test['id'][i]],:])
    except:l_test.append(np.zeros(len(comm_dict)))
l_test=np.vstack(l_test)


In [ ]:
tmp=pd.concat([attr.companyAlias,attr_test.companyAlias],axis=0)
tmp2=pd.concat([attr.ordinal,attr_test.ordinal],axis=0)
tmp3=np.vstack([l,l_test])

data=np.hstack([tmp3,pd.get_dummies(tmp),((tmp2-tmp2.mean())/tmp2.std())[:,np.newaxis]])

X_train=data[:len(attr),:]
X_test=data[len(attr):,:]
y_train=attr.stillExists

In [ ]:
clf=LogisticRegression().fit(X_train,y_train)
print 'training set accuracy:{}'.format(clf.score(X_train,y_train))
pd.DataFrame(clf.predict(X_test)).to_csv('predictions.csv')
